## Paths dataset + carga

In [1]:
product_path = '/home/pablost/Hackathon_inditex_data_science/hackathon-inditex-data-recommender/data/processed/new_processed/products_data.pkl'
user_path = '/home/pablost/Hackathon_inditex_data_science/hackathon-inditex-data-recommender/data/processed/new_processed/user_data.csv'
test_path = '/home/pablost/Hackathon_inditex_data_science/hackathon-inditex-data-recommender/data/processed/new_processed/test_data.pkl'
train_path = '/home/pablost/Hackathon_inditex_data_science/hackathon-inditex-data-recommender/data/processed/new_processed/train_data.pkl'

In [2]:
import pandas as pd

# Cargar los datasets
products = pd.read_pickle(product_path)
users = pd.read_csv(user_path)
test = pd.read_pickle(test_path)
train = pd.read_pickle(train_path)

## Exploration datasets

In [3]:
# Exploración básica
def explore_dataset(dataset, name):
    print(f"Exploración del dataset: {name}")
    print(f"Forma: {dataset.shape}")
    print("Primeras filas:")
    print(dataset.head())
    print("\nTipos de datos:")
    print(dataset.dtypes)
    print("\nValores faltantes:")
    print(dataset.isnull().sum())
    print("-" * 40)

# Llamar a la función para cada dataset
explore_dataset(products, "Products")
explore_dataset(users, "Users")
explore_dataset(test, "Test")
explore_dataset(train, "Train")

Exploración del dataset: Products
Forma: (43692, 6)
Primeras filas:
   discount                                          embedding  partnumber  \
0         0  [-0.13401361, -0.1200429, -0.016117405, -0.167...       32776   
1         0  [-0.0949274, -0.107294075, -0.16559914, -0.174...       41431   
2         0  [-0.12904441, -0.07724628, -0.09799071, -0.164...       39419   
3         1  [-0.12783332, -0.133868, -0.10101265, -0.18888...       36087   
4         1  [-0.14092924, -0.1258284, -0.10809927, -0.1765...       34132   

   color_id  cod_section  family  
0        85            4      73  
1       135            4      73  
2       339            4      73  
3       135            4      73  
4         3            4      73  

Tipos de datos:
discount         int8
embedding      object
partnumber      int32
color_id        int32
cod_section     int16
family          int32
dtype: object

Valores faltantes:
discount       0
embedding      0
partnumber     0
color_id       0
co

#### 1.Tratamiento de usuarios no logueados
`Objetivo`: Confirmar que los valores -1 en user_id se interpreten correctamente como "no logueados".

In [6]:
# Revisar valores únicos de user_id en Train y Test
print("Valores únicos en user_id (Train):", train['user_id'].unique()[:5], "...")
print("Valores únicos en user_id (Test):", test['user_id'].unique()[:5], "...")

# Comprobar cantidad de usuarios no logueados
print("Usuarios no logueados en Train:", (train['user_id'] == -1).sum())
print("Usuarios no logueados en Test:", (test['user_id'] == -1).sum())


Valores únicos en user_id (Train): [    -1 480729 389707 350824  51325] ...
Valores únicos en user_id (Test): [    -1 259465 486272 217706 143895] ...
Usuarios no logueados en Train: 39694715
Usuarios no logueados en Test: 23509


#### 2.Verificar consistencia entre Train y Test
`Objetivo 1:` Asegurar que las columnas clave (user_id, session_id, partnumber) tienen el mismo tipo de datos.

In [7]:
# Verificar tipos de datos
print("Tipos de datos Train:")
print(train.dtypes)
print("\nTipos de datos Test:")
print(test.dtypes)

# Comprobar si los tipos coinciden entre Train y Test
assert train['user_id'].dtype == test['user_id'].dtype, "Tipo de user_id inconsistente"
assert train['session_id'].dtype == test['session_id'].dtype, "Tipo de session_id inconsistente"
assert train['partnumber'].dtype == test['partnumber'].dtype, "Tipo de partnumber inconsistente"
print("Tipos de datos consistentes entre Train y Test.")


Tipos de datos Train:
session_id                  int32
date               datetime64[ns]
timestamp_local    datetime64[ns]
add_to_cart                  int8
user_id                     int32
country                     int16
partnumber                  int32
device_type                  int8
pagetype                    int16
dtype: object

Tipos de datos Test:
session_id                  int32
date               datetime64[ns]
timestamp_local    datetime64[ns]
user_id                     int32
country                     int16
partnumber                  int32
device_type                  int8
pagetype                    int16
dtype: object
Tipos de datos consistentes entre Train y Test.


`Objetivo 2`: Revisar si los partnumber en Test están presentes en Products.

In [8]:
# Verificar si todos los partnumber de Test están en Products
test_partnumbers = test['partnumber'].unique()
products_partnumbers = products['partnumber'].unique()

missing_partnumbers = set(test_partnumbers) - set(products_partnumbers)
print(f"Partnumbers en Test no presentes en Products: {len(missing_partnumbers)}")
if len(missing_partnumbers) > 0:
    print("Ejemplo de Partnumbers faltantes:", list(missing_partnumbers)[:5])

Partnumbers en Test no presentes en Products: 0


#### 3. Validar unicidad
`Objetivo:` Confirmar que los identificadores son únicos.

In [9]:
# Comprobar unicidad de identificadores clave
print("Duplicados en session_id (Train):", train.duplicated(subset=['session_id']).sum())
print("Duplicados en session_id (Test):", test.duplicated(subset=['session_id']).sum())

print("Duplicados en partnumber (Products):", products.duplicated(subset=['partnumber']).sum())


Duplicados en session_id (Train): 41977689
Duplicados en session_id (Test): 21926
Duplicados en partnumber (Products): 0


#### 4. Asegurar alineación con el enfoque
`Colaborativo:` Revisar densidad de interacciones user_id vs. partnumber.

In [10]:
# Comprobar número de interacciones por usuario
interaction_density = train.groupby('user_id')['partnumber'].count()
print("Resumen de densidad de interacciones por usuario:")
print(interaction_density.describe())


Resumen de densidad de interacciones por usuario:
count    3.800530e+05
mean     1.224867e+02
std      6.438882e+04
min      1.000000e+00
25%      3.000000e+00
50%      7.000000e+00
75%      1.900000e+01
max      3.969472e+07
Name: partnumber, dtype: float64


`Contenido:` Validar si los embeddings están normalizados.

In [11]:
# Verificar normalización de embeddings en Products
import numpy as np

embeddings = products['embedding'].apply(lambda x: np.array(x))
norms = embeddings.apply(np.linalg.norm)
print("Estadísticas de normalización de embeddings:")
print(norms.describe())

Estadísticas de normalización de embeddings:
count    43692.000000
mean        10.518237
std          2.610645
min          4.942136
25%          9.917526
50%         11.005746
75%         12.059213
max         19.455055
Name: embedding, dtype: float64


`Popularidad:` Validar tendencias útiles en add_to_cart y pagetype.

In [12]:
# Analizar distribución de add_to_cart y pagetype
print("Distribución de add_to_cart:")
print(train['add_to_cart'].value_counts(normalize=True))
print("\nDistribución de pagetype:")
print(train['pagetype'].value_counts(normalize=True))

Distribución de add_to_cart:
add_to_cart
0    0.941016
1    0.058984
Name: proportion, dtype: float64

Distribución de pagetype:
pagetype
 24    9.936603e-01
 8     2.784876e-03
 6     2.437969e-03
 19    4.196218e-04
 16    3.299361e-04
 7     1.709721e-04
 1     7.845084e-05
 17    4.876326e-05
-1     2.571349e-05
 25    8.957831e-06
 23    6.337075e-06
 20    6.014851e-06
 10    5.069660e-06
 3     4.124469e-06
 5     3.544466e-06
 21    2.942981e-06
 12    1.353341e-06
 13    1.160007e-06
 26    1.052599e-06
 14    7.088931e-07
 9     6.014851e-07
 11    5.800035e-07
 22    2.792609e-07
 15    2.148161e-07
 2     1.074081e-07
 34    4.296322e-08
 31    4.296322e-08
 30    2.148161e-08
 18    2.148161e-08
 4     2.148161e-08
 32    2.148161e-08
 29    2.148161e-08
 37    2.148161e-08
 27    2.148161e-08
 33    2.148161e-08
 35    2.148161e-08
 36    2.148161e-08
 28    2.148161e-08
Name: proportion, dtype: float64


### Ajustes sobre los datasets para que se alinen mejor a nuestro enfoque:

In [14]:
# Filtrar duplicados en Train por session_id y partnumber
duplicates = train[train.duplicated(subset=['session_id', 'partnumber'], keep=False)]
print("Ejemplo de duplicados en Train:")
print(duplicates.head())

# Analizar diferencias entre duplicados
differences = duplicates.groupby(['session_id', 'partnumber']).nunique()
print("Número de valores únicos por columna en duplicados:")
print(differences)


Ejemplo de duplicados en Train:
    session_id       date         timestamp_local  add_to_cart  user_id  \
4         1220 2024-06-04 2024-06-04 08:21:13.476            0   480729   
5         1220 2024-06-04 2024-06-04 08:21:09.139            0   480729   
9         1222 2024-06-13 2024-06-13 06:17:57.411            0       -1   
11        1222 2024-06-13 2024-06-13 06:19:09.272            1       -1   
13        1222 2024-06-13 2024-06-13 06:19:18.028            0       -1   

    country  partnumber  device_type  pagetype  
4        25        1592            1        24  
5        25        1592            1        24  
9        57        5249            1        24  
11       57        5249            1        24  
13       57       21714            1        24  


#### **Análisis de los Resultados de los ajustes**

##### **1. Duplicados en `Train`**
**Observaciones**:
- Los duplicados tienen valores idénticos en algunas columnas (`user_id`, `country`, `partnumber`, `device_type`, `pagetype`) pero difieren en:
  - **`timestamp_local`**: Marca temporal diferente, indicando que son interacciones distintas en la misma sesión.
  - **`add_to_cart`**: En un caso el producto fue añadido al carrito (`1`), en otro no (`0`).

**Implicaciones**:
- Estos datos duplicados no son redundantes; reflejan interacciones diferentes dentro de la misma sesión. Por ejemplo:
  - Para `session_id = 1222`, hay dos interacciones con `partnumber = 5249` con diferentes acciones (`add_to_cart = 0` y `add_to_cart = 1`).
  - Este tipo de duplicado es **información válida** y útil para capturar el comportamiento del usuario.

**Recomendaciones**:
1. **No eliminar duplicados directamente**: Eliminar duplicados puede significar pérdida de información sobre las acciones del usuario.
2. **Manejar duplicados en función del modelo**:
   - Si usamos modelos **basados en sesiones**, podemos priorizar la interacción más reciente (`timestamp_local`) o con mayor relevancia (`add_to_cart = 1`).
   - Si usamos modelos **basados en todas las interacciones**, conservar todas las filas para capturar el comportamiento completo del usuario.

In [16]:
# Priorizar filas con add_to_cart = 1 dentro de cada session_id y partnumber
train = train.sort_values(by=['add_to_cart', 'timestamp_local'], ascending=[False, False])
train = train.drop_duplicates(subset=['session_id', 'partnumber'], keep='first')

print(f"Forma del dataset después de priorizar interacciones relevantes: {train.shape}")


Forma del dataset después de priorizar interacciones relevantes: (36858893, 9)


In [15]:
# Normalizar embeddings
products['embedding'] = products['embedding'].apply(lambda x: (np.array(x) / np.linalg.norm(x)).tolist())
print("Normalización completada. Ejemplo de embeddings normalizados:")
print(products['embedding'].head())


Normalización completada. Ejemplo de embeddings normalizados:
0    [-0.01306566409766674, -0.011703588999807835, ...
1    [-0.009752826765179634, -0.011023376137018204,...
2    [-0.012108728289604187, -0.00724831223487854, ...
3    [-0.0129902558401227, -0.01360349077731371, -0...
4    [-0.01368741039186716, -0.012220778502523899, ...
Name: embedding, dtype: object


#### Sustituimos valores -1 de pagetype, device_type y country. (en RAW eran null)

In [21]:
# Variables categóricas: Reemplazar -1 por 'missing'
cols_categorical = ['pagetype', 'device_type']
for col in cols_categorical:
    train[col] = train[col].replace(-1, 'missing')
    test[col] = test[col].replace(-1, 'missing')

# Variables numéricas: Imputar valores faltantes
train['country'] = train['country'].replace(-1, train['country'].mode()[0])
test['country'] = test['country'].replace(-1, test['country'].mode()[0])

# Confirmar cambios
print("Cambios realizados:")
print(train[['pagetype', 'device_type', 'country']].head())


Cambios realizados:
         pagetype  device_type  country
10725642       24            3       29
35628113       24            1       34
35628118       24            1       34
9475628        24            3       34
9475555        24            3       34


---

## Muestreo para el entrenamiento - 1% inicialmente

In [22]:
# Muestreo de Train (1% del dataset para pruebas)
train_sample = train.sample(frac=0.01, random_state=42)
print(f"Tamaño de la muestra de Train: {train_sample.shape}")

Tamaño de la muestra de Train: (368589, 9)


##### Validación

In [24]:
# Confirmar el tamaño de la muestra
print(f"Tamaño esperado de la muestra: {len(train) * 0.01:.0f} filas")


Tamaño esperado de la muestra: 368589 filas


### 1. Explorar la muestra
Revisar la distribución y características de la muestra para confirmar que representa adecuadamente el dataset completo:

In [19]:
# Exploración básica de la muestra
print(train_sample.describe())
print(train_sample['add_to_cart'].value_counts(normalize=True))


         session_id                           date  \
count  3.685890e+05                         368589   
mean   2.585259e+06  2024-06-07 07:01:04.667149568   
min    1.000000e+01            2024-06-01 00:00:00   
25%    1.290344e+06            2024-06-04 00:00:00   
50%    2.583594e+06            2024-06-07 00:00:00   
75%    3.879591e+06            2024-06-11 00:00:00   
max    5.171732e+06            2024-06-15 00:00:00   
std    1.494213e+06                            NaN   

                     timestamp_local    add_to_cart        user_id  \
count                         368589  368589.000000  368589.000000   
mean   2024-06-07 22:01:48.314542336       0.071741   41608.927404   
min       2024-06-01 02:00:03.965000       0.000000      -1.000000   
25%       2024-06-04 10:10:05.748000       0.000000      -1.000000   
50%    2024-06-07 19:54:29.023000064       0.000000      -1.000000   
75%       2024-06-11 08:20:13.328000       0.000000      -1.000000   
max       2024-06-15 11

In [25]:
# Exploración de la muestra ajustada
print(train_sample.describe())
print(train_sample['pagetype'].value_counts(normalize=True))
print(train_sample['device_type'].value_counts(normalize=True))
print(train_sample['country'].value_counts(normalize=True))


         session_id                           date  \
count  3.685890e+05                         368589   
mean   2.585259e+06  2024-06-07 07:01:04.667149568   
min    1.000000e+01            2024-06-01 00:00:00   
25%    1.290344e+06            2024-06-04 00:00:00   
50%    2.583594e+06            2024-06-07 00:00:00   
75%    3.879591e+06            2024-06-11 00:00:00   
max    5.171732e+06            2024-06-15 00:00:00   
std    1.494213e+06                            NaN   

                     timestamp_local    add_to_cart        user_id  \
count                         368589  368589.000000  368589.000000   
mean   2024-06-07 22:01:48.314542336       0.071741   41608.927404   
min       2024-06-01 02:00:03.965000       0.000000      -1.000000   
25%       2024-06-04 10:10:05.748000       0.000000      -1.000000   
50%    2024-06-07 19:54:29.023000064       0.000000      -1.000000   
75%       2024-06-11 08:20:13.328000       0.000000      -1.000000   
max       2024-06-15 11

## Dividir la muestra en conjuntos de entrenamiento y validación
Preparar los datos para probar modelos:

In [26]:
from sklearn.model_selection import train_test_split

# Dividir la muestra
train_set, val_set = train_test_split(train_sample, test_size=0.2, random_state=42)
print(f"Tamaño del conjunto de entrenamiento: {train_set.shape}")
print(f"Tamaño del conjunto de validación: {val_set.shape}")

Tamaño del conjunto de entrenamiento: (294871, 9)
Tamaño del conjunto de validación: (73718, 9)


### Análisis

#### **1. Distribución temporal**
- **`date` y `timestamp_local`**:
  - La distribución temporal sigue siendo consistente y cubre del **2024-06-01** al **2024-06-15**.
  - **Media, percentiles y rango**: Indican una buena representación del rango temporal de interacciones.

**Conclusión**: La muestra refleja adecuadamente las interacciones del período completo.

---

#### **2. Variable `add_to_cart`**
- **Proporción**:
  - `0`: **92.8%**.
  - `1`: **7.2%**.
- La proporción es consistente con la muestra anterior, confirmando que los cambios no afectaron la representación del desbalance.

**Conclusión**: El desbalance sigue presente y debe manejarse adecuadamente en el modelado.

---

#### **3. Variables categóricas**
##### **`pagetype`**
- **Distribución**:
  - Dominancia del valor `24` (**99.2%**), lo que indica que es la página más interactuada.
  - La categoría `missing` representa solo el **0.0022%**, indicando un impacto mínimo en la distribución.
  
##### **`device_type`**
- **Distribución**:
  - `1` (probablemente móvil o escritorio) domina con **92.2%**.
  - `3` y `2` representan una proporción menor pero significativa.

**Conclusión**: 
- Ambas variables tienen distribuciones consistentes, y la categoría `missing` no afecta significativamente el equilibrio.

---

#### **4. Variable numérica `country`**
- **Distribución**:
  - Los países `29`, `57`, `34`, y `25` representan la totalidad de la muestra.
  - El valor imputado (probablemente uno de estos países) se integra bien en la distribución global.

**Conclusión**: La imputación en `country` fue exitosa y no distorsiona la representación de los datos.

---

#### **2. División en conjuntos de entrenamiento y validación**
- **Tamaño del entrenamiento**: **294,871 filas** (80% de la muestra).
- **Tamaño de la validación**: **73,718 filas** (20% de la muestra).
- **Implicaciones**:
  - La división parece adecuada para experimentos iniciales, dejando un conjunto representativo para evaluar el rendimiento del modelo.


---

## Persistimos los datasets (Back_up)

In [27]:
# Definir rutas para guardar los datasets
train_path = "/home/pablost/Hackathon_inditex_data_science/hackathon-inditex-data-recommender/data/processed/hybrid_model/train_preprocessed.pkl"
test_path = "/home/pablost/Hackathon_inditex_data_science/hackathon-inditex-data-recommender/data/processed/hybrid_model/test_preprocessed.pkl"
sample_path = "/home/pablost/Hackathon_inditex_data_science/hackathon-inditex-data-recommender/data/processed/hybrid_model/train_sample_preprocessed.pkl"
train_set_path = "/home/pablost/Hackathon_inditex_data_science/hackathon-inditex-data-recommender/data/processed/hybrid_model/train_set.pkl"
val_set_path = "/home/pablost/Hackathon_inditex_data_science/hackathon-inditex-data-recommender/data/processed/hybrid_model/val_set.pkl"

# Guardar los datasets
train.to_pickle(train_path)
test.to_pickle(test_path)
train_sample.to_pickle(sample_path)
train_set.to_pickle(train_set_path)
val_set.to_pickle(val_set_path)

print("Todos los datasets han sido guardados correctamente.")


Todos los datasets han sido guardados correctamente.


---

## -